In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}]/{size:>5d}")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
------------------------------------------
loss: 2.295082 [   64]/60000
loss: 2.284985 [ 6464]/60000
loss: 2.266774 [12864]/60000
loss: 2.263555 [19264]/60000
loss: 2.238910 [25664]/60000
loss: 2.211938 [32064]/60000
loss: 2.218142 [38464]/60000
loss: 2.180317 [44864]/60000
loss: 2.187547 [51264]/60000
loss: 2.152300 [57664]/60000
Test Error: 
 Accuracy: 48.4%, Avg loss: 2.143292 

Epoch 2
------------------------------------------
loss: 2.156031 [   64]/60000
loss: 2.141013 [ 6464]/60000
loss: 2.080255 [12864]/60000
loss: 2.097712 [19264]/60000
loss: 2.039030 [25664]/60000
loss: 1.981805 [32064]/60000
loss: 2.007349 [38464]/60000
loss: 1.926623 [44864]/60000
loss: 1.936944 [51264]/60000
loss: 1.859933 [57664]/60000
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.855886 

Epoch 3
------------------------------------------
loss: 1.889614 [   64]/60000
loss: 1.851445 [ 6464]/60000
loss: 1.731836 [12864]/60000
loss: 1.776674 [19264]/60000
loss: 1.663863 [25664]/60000
loss: 1.621864 [32